<a href="https://colab.research.google.com/github/swethamohan1/MultiClass-Image-Classification/blob/master/PR_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import random
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout,Lambda
from keras.layers import SeparableConv2D,BatchNormalization,MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
# from google.colab import drive
# drive.mount('/content/drive')
# from zipfile import ZipFile
# with ZipFile('./drive/My Drive/seg_test.zip', 'r') as zipObj:
#    # Extract all the contents of zip file in current directory
#    zipObj.extractall()
# with ZipFile('./drive/My Drive/seg_train.zip', 'r') as zipObj:
#    # Extract all the contents of zip file in current directory
#    zipObj.extractall()

train_dir = './seg_train/'
test_dir = './seg_test/'

datagen = ImageDataGenerator( rescale=1.0/255.,
                                    shear_range=0.2,  
                                    zoom_range=0.2,        
                                    horizontal_flip=True,
                                    validation_split=0.3)
train_gen= datagen.flow_from_directory(train_dir,
                                        batch_size=8,
                                        class_mode='categorical',
                                        target_size=(150, 150))

test_gen = datagen.flow_from_directory(test_dir,
                                        batch_size=8,
                                        class_mode='categorical',
                                        target_size=(150, 150))

def CNN_arch():
    model = Sequential()
    model.add(Lambda(lambda x: x, input_shape=(150, 150, 3)))

    model.add(SeparableConv2D(16, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(SeparableConv2D(32, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(SeparableConv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(SeparableConv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Flatten())
    model.add(Dense(1024,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6,activation='sigmoid'))
    return model

model = CNN_arch()
model.summary()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_6 (Lambda)            (None, 150, 150, 3)       0         
_________________________________________________________________
separable_conv2d_13 (Separab (None, 148, 148, 16)      91        
_________________________________________________________________
batch_normalization_21 (Batc (None, 148, 148, 16)      64        
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 74, 74, 16)        0         
_________________________________________________________________
separable_conv2d_14 (Separab (None, 72, 72, 32)        688       
______________________________________

In [ ]:
opt = SGD(lr=1e-4,momentum=0.99)

model.compile(
    loss='binary_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)
    
history = model.fit_generator(
    train_gen, 
    steps_per_epoch  = 150, 
    validation_data  = test_gen,
    validation_steps = 150,
    epochs = 5, 
    verbose = 1
)
model_score = model.evaluate_generator(test_gen)
print("Model Test Loss:",model_score[0])
print("Model Test Accuracy:",model_score[1])

Epoch 1/5
150/150 [==============================] - 14s 93ms/step - loss: 0.3139 - accuracy: 0.8719 - val_loss: 0.3090 - val_accuracy: 0.8361
Epoch 2/5
150/150 [==============================] - 13s 89ms/step - loss: 0.3172 - accuracy: 0.8651 - val_loss: 0.4884 - val_accuracy: 0.8861
Epoch 3/5
150/150 [==============================] - 13s 90ms/step - loss: 0.2889 - accuracy: 0.8790 - val_loss: 0.3462 - val_accuracy: 0.8649
Epoch 4/5
150/150 [==============================] - 13s 88ms/step - loss: 0.2948 - accuracy: 0.8772 - val_loss: 0.1696 - val_accuracy: 0.8889
Epoch 5/5
150/150 [==============================] - 13s 88ms/step - loss: 0.2947 - accuracy: 0.8772 - val_loss: 0.4070 - val_accuracy: 0.8712
Model Test Loss: 0.3967212438583374
Model Test Accuracy: 0.8723332285881042
